In [1]:
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout


/Users/giselyalves/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
filename='/Users/giselyalves/Downloads/yelp_review.csv'
data = pd.read_csv(filename)[:15000]
data['sentiment'] = [ 'pos' if (x>3) else 'neutral' if (x == 3) else 'neg' for x in data['stars']]

In [3]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
tokenizer = Tokenizer(num_words=2500, lower=True,split=' ')
tokenizer.fit_on_texts(data['text'].values)


In [4]:
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X) #adjusts the size of the sequences

In [5]:
embed_dim = 128
lstm_out = 300
batch_size= 32

##Buidling the LSTM network

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1]))#transform each word in integer
model.add(LSTM(lstm_out))
model.add(Dropout(0.6))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [6]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

#Here we train the Network.

model.fit(X_train, Y_train, batch_size=batch_size, epochs = 1, verbose = 1, validation_split=0.1)

# Measuring score and accuracy on validation set
score,acc = model.evaluate(X_test, Y_test, verbose = 0, batch_size = batch_size)
print("Loss score: %.2f" % (score))
print("Test Accuracy: %.2f" % (acc*100))


Train on 10800 samples, validate on 1200 samples
Epoch 1/1
10800/10800 [==============================] - 1334s 124ms/step - loss: 0.7541 - acc: 0.6944 - val_loss: 0.6426 - val_acc: 0.7492
Loss score: 0.61
Test Accuracy: 75.37
